# РАЗРАБОТКА КЛАССА ПОЛЬЗОВАТЕЛЬСКОГО ПРОСТРАНСТВА ЭМБЕДДИНГОВ

## 1) Импорт используемых библиотек

Сначала необходимо инпортировать все библиотеки, которые будут использоваться при разработке класса `UES`.

Для работы с содержимым используемого Google Диска через платформу Google Colab требуется импортировать следующие библиотеки:
- `drive` - модуль, который позволяет подключить Google Диск к виртуальной машине среды выполнения и использовать его содержимое.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Далее импортируем библиотеки, используемые в дальнейшем:
- `pandas` - библиотека для обработки и анализа структурированных данных;
- `numpy` - библиотека, которую применяют для математических вычислений: начиная с базовых функций и заканчивая линейной алгеброй;
- `re` - библиотека, предоставляющая мощные инструменты для работы с текстом.
- `gensim` - библиотека обработки естественного языка предназначения для «Тематического моделирования»;
- `sklearn` - библиотека, реализующая методы машинного обучения, в состав которой входят различные алгоритмы, в том числе предназначенные для задач классификации, регрессионного и кластерного анализа данных, включая метод опорных векторов, метод случайного леса, алгоритм усиления градиента, метод k-средних и DBSCAN;
- `plotly` - графическая библиотека, которая позволяется создавать интерактивные графики.;
- `wordcloud` - библиотека, с помощью которой реализуется метод визуализации данных облако слов, используемый для представления текстовых данных, в котором размер каждого слова указывает на его частоту или важность.

In [ ]:
pip install scikit-learn-extra

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 9.9 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd

import re

from gensim.parsing.preprocessing import preprocess_string
from gensim.models.doc2vec import Doc2Vec

from sklearn.manifold import TSNE
from sklearn_extra.cluster import KMedoids

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from wordcloud import WordCloud

## 2) Подготовка используемых ресурсов

Для начала определим корпус книжных аннотаций, датасет пользовательских предпочтений и модель Doc2Vec, ранее обученную на корпуса аннотаций:

In [ ]:
df_user_data = pd.read_csv(
    filepath_or_buffer='/content/drive/MyDrive/ВКР/children/children_user_data.csv',
    index_col='user_id'
)

In [ ]:
df_corpus_annotations = pd.read_csv(
    filepath_or_buffer='/content/drive/MyDrive/ВКР/children/children_corpus_annotations.csv',
    index_col='book_id'
)

In [ ]:
filename_d2v = '/content/drive/MyDrive/ВКР/children/children_model_d2v.d2v'
model_d2v = Doc2Vec.load(filename_d2v)

VECTOR_SIZE = model_d2v.vector_size
print(f"Размерность формируемых векторов - {VECTOR_SIZE}")

Размерность формируемых векторов - 200


## 3) Корректирование исходных эмбеддингов

Определим категории взаимодействий пользователя с книгами:

In [ ]:
READ     = 'read'
SHELVED  = 'shelved'
RATING_0 = 'rating_0'
RATING_1 = 'rating_1'
RATING_2 = 'rating_2'
RATING_3 = 'rating_3'
RATING_4 = 'rating_4'
RATING_5 = 'rating_5'

interactions = [SHELVED, READ, RATING_0, RATING_1, RATING_2, RATING_3, RATING_4, RATING_5]

print('Виды реакций пользователей:')
print(f"1) Прочитанные книги                   - {READ}")
print(f"2) Сохраненны книги, но не прочитанные - {SHELVED}")
print(f"3) Книги с оценкой 0                   - {RATING_0}")
print(f"4) Книги с оценкой 1                   - {RATING_1}")
print(f"5) Книги с оценкой 2                   - {RATING_2}")
print(f"6) Книги с оценкой 3                   - {RATING_3}")
print(f"7) Книги с оценкой 4                   - {RATING_4}")
print(f"8) Книги с оценкой 5                   - {RATING_5}")

Виды реакций пользователей:
1) Прочитанные книги                   - read
2) Сохраненны книги, но не прочитанные - shelved
3) Книги с оценкой 0                   - rating_0
4) Книги с оценкой 1                   - rating_1
5) Книги с оценкой 2                   - rating_2
6) Книги с оценкой 3                   - rating_3
7) Книги с оценкой 4                   - rating_4
8) Книги с оценкой 5                   - rating_5


Проанализировав множество эмбеддингов, формируемых при помощи метода `infer_vector()` модели Doc2Vec, и научную работу, в которой предлагается данная модель, можем прийти к выводу о том, что значения компонент эмбеддингов являются нормализованными, то есть варьируются от -1 до 1. Таким образом, при добавлении дополнительной размерности, учитывающей характер взаимодействия пользователя, следует взять значения, лежащие в том же диапазоне.

In [ ]:
left_boundary = -1
right_boundary = 1
step = (right_boundary - left_boundary) / (len(interactions) - 1)

VALUE_READ     = left_boundary + 0 * step
VALUE_SHELVED  = left_boundary + 1 * step
VALUE_RATING_0 = left_boundary + 2 * step
VALUE_RATING_1 = left_boundary + 3 * step
VALUE_RATING_2 = left_boundary + 4 * step
VALUE_RATING_3 = left_boundary + 5 * step
VALUE_RATING_4 = left_boundary + 6 * step
VALUE_RATING_5 = left_boundary + 7 * step
VALUE_DEFAULT  = np.nan

print(f"Характеристики формирования дополнительной компоненты эмбеддинга:")
print(f"- Левая граница диапазона         - {left_boundary}")
print(f"- Правая граница диапазона        - {right_boundary}")
print(f"- Количество реакций пользователя - {len(interactions)}")
print(f"- Шаг формирования значения       - {step}\n")

print(f"Значения дополнительной компоненты эмбеддинга:")
print(f"1) Категория «{READ}»      - {VALUE_READ}")
print(f"2) Категория «{SHELVED}»   - {VALUE_SHELVED}")
print(f"3) Категория «{RATING_0}»  - {VALUE_RATING_0}")
print(f"4) Категория «{RATING_1}»  - {VALUE_RATING_1}")
print(f"5) Категория «{RATING_2}»  - {VALUE_RATING_2}")
print(f"6) Категория «{RATING_3}»  - {VALUE_RATING_3}")
print(f"7) Категория «{RATING_4}»  - {VALUE_RATING_4}")
print(f"8) Категория «{RATING_5}»  - {VALUE_RATING_5}")
print(f"9) Значение по умолчанию - {VALUE_DEFAULT}")

Характеристики формирования дополнительной компоненты эмбеддинга:
- Левая граница диапазона         - -1
- Правая граница диапазона        - 1
- Количество реакций пользователя - 8
- Шаг формирования значения       - 0.2857142857142857

Значения дополнительной компоненты эмбеддинга:
1) Категория «read»      - -1.0
2) Категория «shelved»   - -0.7142857142857143
3) Категория «rating_0»  - -0.4285714285714286
4) Категория «rating_1»  - -0.1428571428571429
5) Категория «rating_2»  - 0.1428571428571428
6) Категория «rating_3»  - 0.4285714285714284
7) Категория «rating_4»  - 0.7142857142857142
8) Категория «rating_5»  - 1.0
9) Значение по умолчанию - nan


## 4) Разработка класса UES

Определим класс   `UES` (User Embedding Space), который описывает пространство эмбеддингов конкретного пользователя. Каждый экземпляр содержит в себе набор эмбеддингов, учитывающих, как и объект, так и вид взаимодействия с ним, и набор средств анализа рассчитанного пространства.

In [ ]:
class UES(object):
    '''
    USER EMBEDDING SPACE

    Класс, который описывает пространство эмбеддингов конкретного пользователя.
    Экземпляр содержит в себе набор эмбеддингов, учитывающих, как и объект,
    так и вид взаимодействия с ним, и набор средств анализа рассчитанного
    пространства.
    '''


    def __init__(self, user_data):
        '''
        Метод инициализации класса, в котором производится обращение к методам,
        формирующим собственно пространство эмбеддингов.

        Аргументы:
        - user_data  (`pandas.Series`) - серия библиотеки Pandas, в которой
            содержится препроцессированная информация о взаимодействиях
            пользователя с книгами в виде категоризарованного набора
            идентификаторов книг.
        '''

        self.user_id = user_data.name
        # Датасет, в котором хранится все рассчитанные данные
        self.df = pd.DataFrame(columns=['id', 'embedding', 'interaction', 'type', 'tokens'])
        # Формирование пространства эмбеддингов
        self.__construct_embedding_space(user_data)
        self.__compute_centers()


    def __construct_embedding_space(self, user_data):
        '''
        Метод, при помощи которого происходит расчёт эмбеддингов: формирование
        эмбеддинга с помощью модели Doc2Vec и его последующая корректировка.

        Аргументы:
        - user_data  (`pandas.Series`) - серия библиотеки Pandas, в которой
            содержится препроцессированная информация о взаимодействиях
            пользователя с книгами в виде категоризарованного набора
            идентификаторов книг.
        '''

        for interaction in interactions:
            # Идентификаторы книг текущего вида взаимодейсвия
            ids = re.findall('[0-9]+', user_data[interaction])
            for id in ids:
                # Если имеется аннотация рассматриваемой, происходит формирование эмбеддинга
                try:
                    # Формирование изначального эмбеддинга
                    text_tokens = preprocess_string(df_corpus_annotations.loc[int(id)]['annotation'])
                    text_embedding = model_d2v.infer_vector(text_tokens)
                    # Добавление измерения, характеризующего вид взаимодействия
                    interaction_embedding = np.append(text_embedding, self.__get_interaction_value(interaction))
                    # Запись полученного эмбеддинга в датасет пространства
                    new_row = {'id': f"{interaction}_{id}",
                               'embedding': interaction_embedding,
                               'interaction': interaction,
                               'type': 'object',
                               'tokens': ' '.join(text_tokens)}
                    self.df.loc[len(self.df)] = new_row
                except:
                    continue
        self.df = self.df.set_index('id')


    def __get_interaction_value(self, interaction):
        '''
        Метод, который предоставляет значение дополнительной компоненты
        эмбеддинга в зависимости от вида взаимодействия пользователя с книгой.

        Аргументы:
        - interaction  (`str`) - вид взаимодействия пользователя с книгой.
        '''

        match interaction:
            case 'shelved':
                return VALUE_SHELVED
            case 'read':
                return VALUE_READ
            case 'rating_0':
                return VALUE_RATING_0
            case 'rating_1':
                return VALUE_RATING_1
            case 'rating_2':
                return VALUE_RATING_2
            case 'rating_3':
                return VALUE_RATING_3
            case 'rating_4':
                return VALUE_RATING_4
            case 'rating_5':
                return VALUE_RATING_5
            case _:
                return VALUE_DEFAULT


    def __compute_centers(self):
        '''
        Метод, позволяющий получить центры (медоиды) кластеров взаимодействий
        при помощи алгоритма K-Medoids.

        Все вычисленные центры кластеров помечаются в датасете `self.df`.
        '''

        for interaction in interactions:
            # Элементы кластера текущего вида взаимодействия
            interaction_data = self.df[self.df['interaction'] == interaction]
            embeddings = interaction_data['embedding'].values.tolist()
            # Если текущий кластер не является пустым, происходит поиск его центра
            if not (len(embeddings) == 0):
                # Поиск центра кластера при помощи алгоритма K-Means
                kmedoids = KMedoids(n_clusters=1)
                kmedoids.fit(embeddings)
                center = kmedoids.cluster_centers_[0]
                # Отметка центра в датасете пространства
                for index, row in interaction_data.iterrows():
                    if (np.array_equal(row['embedding'], center)):
                        self.df.at[index, 'type'] = 'center'
                        break


    def retrieve_centers(self):
        '''
        Метод, возвращающий раннее вычисленные центры кластеров взаимодействий
        в виде фрагмента датасета.

        Возвращает:
        - df (`pandas.DataFrame`) - датасет с центрами кластеров.
        '''

        return self.df[self.df['type'] == 'center']


    def show_tsne(self, is_text_embeddings=False):
        '''
        Метод, при помощи которого происходит визуализация пространства
        эмбеддингов пользователя сниженной размерности при помощи
        алгоритма T-SNE.

        Аргументы:
        - is_text_embeddings (`bool`) - если аргумент равен `True`, то происходит
            визуализация исходного пространства эмбеддингов, иначе -
            пространства эмбеддингов с учётом вида взаимодействия.
        '''

        # Размерность эмбеддингов очределяет учёт взаимодействий
        embedding_size = VECTOR_SIZE if (is_text_embeddings) else VECTOR_SIZE + 1
        # Формирование итоговых эмбеддингов
        embeddings = np.zeros((len(self.df['embedding']), embedding_size))
        for i in range(len(self.df['embedding'])):
            embeddings[i,:] = np.array(
                self.df.iloc[i]['embedding'][:embedding_size]
            ).reshape((1, embedding_size))
        # Уменьшение размерности при помощи модели T-SNE
        perplexity = len(embeddings) - 2 if (len(embeddings) < 50) else 30
        model_tsne = TSNE(
            perplexity=perplexity, n_components=3, init='pca'
        )

        # Сохрание полученных значений в датасет пространства
        tsne_embeddings = model_tsne.fit_transform(embeddings)
        tsne_x = list(map(lambda x: x[0], tsne_embeddings))
        tsne_y = list(map(lambda x: x[1], tsne_embeddings))
        tsne_z = list(map(lambda x: x[2], tsne_embeddings))
        if (is_text_embeddings):
            self.df['tsne_x_text'] = tsne_x
            self.df['tsne_y_text'] = tsne_y
            self.df['tsne_z_text'] = tsne_z
        else:
            self.df['tsne_x_interaction'] = tsne_x
            self.df['tsne_y_interaction'] = tsne_y
            self.df['tsne_z_interaction'] = tsne_z

        # Визуализация полученных результатов
        if (is_text_embeddings):
            title = 'Визуализация UES при помощи метода T-SNE<br>(без учёта взаимодействий)'
            x = 'tsne_x_text'
            y = 'tsne_y_text'
            z = 'tsne_z_text'
        else:
            title = 'Визуализация UES при помощи метода T-SNE<br>(с учётом взаимодействий)'
            x = 'tsne_x_interaction'
            y = 'tsne_y_interaction'
            z = 'tsne_z_interaction'
        # Построение 3-ёх мерной диаграммы рассеяния
        fig_1 = px.scatter_3d(
            self.df, x=x, y=y, z=z, color='interaction', symbol='type',
            labels={'interaction': 'Вид взаимодействия', 'type': 'Тип объекта'}
        )
        fig_1.update_layout(
            height=470, width=800, title=dict(text=title, font=dict(size=18)),
            scene=dict(xaxis_title='X', yaxis_title='Y', zaxis_title='Z')
        )
        fig_1.show()
        # Построение матрицы рассеяния
        fig_2 = px.scatter_matrix(
            self.df, dimensions=[x, y, z], color='interaction',
            labels={'interaction': "Вид взаимодействия", x: 'X', y: 'Y', z: 'Z'}
        )
        fig_2.update_layout(
            height=470, width=800,
            title=dict(text='Матрица диаграмм рассеяния', font=dict(size=18))
          )
        fig_2.show()


    def show_wordclouds(self):
        '''
        Метод, визуализирующий кластеры взаимодействий посредством
        формирования облака слов.
        '''

        fig = make_subplots(
            rows=3, cols=3,
            subplot_titles=('Shelved', 'Read', 'Rating - 0', 'Rating - 1',
                            'Rating - 2', 'Rating - 3', 'Rating - 4', 'Rating - 5')
        )
        index = 0
        for interaction in interactions:
            row_index = int(index / 3 + 1)
            col_index = int(index % 3 + 1)
            if not (len(self.df[self.df['interaction'] == interaction]) == 0):
                # Если текущий кластер не пустой, происходит построение облака слов
                text = ' '.join(self.df[self.df['interaction'] == interaction]['tokens'])
                wordcloud = WordCloud(
                    width=500, height=500, min_font_size=10, background_color="white"
                ).generate(text)
                fig.add_trace(px.imshow(wordcloud).data[0], row=row_index, col=col_index)
                fig.update_xaxes(visible=False, row=row_index, col=col_index)
                fig.update_yaxes(visible=False, row=row_index, col=col_index)
            else:
                # Если текущий кластер пустой, происходит вывод пустого облака слов
                wordcloud = WordCloud(
                    width=500, height=500, min_font_size=10, background_color="white"
                ).generate('Empty')
                fig.add_trace(px.imshow(wordcloud).data[0], row=row_index, col=col_index)
                fig.update_xaxes(visible=False, row=row_index, col=col_index)
                fig.update_yaxes(visible=False, row=row_index, col=col_index)
            index += 1
        fig.update_layout(height=900, width=850,
                          title=dict(text='Облака слов в зависимости от реакции пользователя',
                                     font=dict(size=18)))
        fig.show()

## 5)	Анализ работы класса UES

Далее произведём анализ работы разработанного класса, рассчитав пространство эмбеддингов для пользователя с идентификатором `8842281e1d1347389f2ab93d60773d4d`.

In [ ]:
df_user_data.iloc[20:30]

,read,shelved,rating_0,rating_1,rating_2,rating_3,rating_4,rating_5
user_id,,,,,,,,
7b2e5fe9fd353fecf3eeebb4850b88d3,[30653713 24612624 22064780 20575434 17456988 ...,[],[],[],[],[24612624 18079564 17349055 18393324 17349203 ...,[30653713 22064780 20575434 17456988 18651970 ...,[20322044 16101018 37186 11594337 10806008 ...
bafc2d50014200cda7cb2b6acd60cd73,[ 240130 17349203 5 37732 3636 ...,[],[],[],[125507],[32929],[113946 6689 7788 90072 232576 6310 23...,[ 240130 17349203 5 37732 3636 ...
9a6f991d0c99a4df68d01a85191d6184,[ 4948 3636 157993 2998],[60177],[],[],[],[],[157993],[4948 3636 2998]
4fe16e3bcbbce3f8801ac6924217fd3b,[378 5],[],[],[],[5],[],[378],[]
83d6e6f80d7c32c6676b3ab3b01543cd,[3636],[],[],[],[],[3636],[],[]
3ca7375dba942a760e53b726c472a7dd,[26875588 857445 766955 767680 1099301 ...,[46306 37190 83369],[],[],[3636],[444304 401679],[26875588 1099301 197084 5 23772 ...,[857445 766955 767680 293595 32929 4948]
0ef32090550901ead25cb0ea21c4d36b,[22671451 20670372 157993 24178 15715080 ...,[33158525 33016249 627821 727364 20949046 ...,[],[],[],[20670372 12217784 9965191],[22671451 157993 24178 15715080 224926 ...,[ 156806 93380 77767 761365 1325218 371136]
0b9a0d35734107c5df4a1e3787193afb,[140225 5],[],[],[],[140225],[],[5],[]
93c5e16254e7838b69178338bb20459e,[ 1852 196970 90072 420282 197084 1139...,[3636],[],[],[],[ 90072 197084 1325218],[ 1852 196970 420282 24178 21348 46306 22...,[113946]


В результате создания экземпляра класса `UES` рассчитывается пользовательское пространство эмбеддингов, данные которого хранятся в датасете `df`:

In [ ]:
ues = UES(df_user_data.loc['8842281e1d1347389f2ab93d60773d4d'])
ues.df.head()

,embedding,interaction,type,tokens
id,,,,
shelved_10893214,"[0.005115694832056761, 0.00019861468172166497,...",shelved,object,cavendish home boi girl definit learn lesson a...
shelved_33282947,"[0.0029936579521745443, 0.007781986612826586, ...",shelved,object,year old alex petroski love space rocket mom b...
shelved_11387515,"[0.002114801201969385, 0.018781716004014015, 0...",shelved,object,won look like think probabl wor august auggi p...
shelved_24396144,"[-0.0020668748766183853, 0.0037662633694708347...",shelved,object,pierr maze detect new case stolen maze stone p...
shelved_20484662,"[-0.004784373566508293, -0.011810575611889362,...",shelved,object,juni jone ivi bean come lovabl energet littl s...


In [ ]:
ues.df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 126 entries, read_30653713 to rating_5_12467482
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   embedding    126 non-null    object
 1   interaction  126 non-null    object
 2   type         126 non-null    object
 3   tokens       126 non-null    object
dtypes: object(4)
memory usage: 9.0+ KB


Вслед за этим произведём выгрузку всех центров кластеров взаимодействий и получим следующий датасет:

In [ ]:
ues.retrieve_centers()

,embedding,interaction,type,tokens
id,,,,
shelved_809849,"[-0.0010988630820065737, 0.0035453320015221834...",shelved,center,child write zoo pet zoo send seri unsuit pet r...
read_42407,"[0.0005407451535575092, 0.002594402525573969, ...",read,center,babar establish celestevil beauti happi citi q...
rating_4_235324,"[0.0014150608330965042, 1.8569533494883217e-05...",rating_4,center,littl sophi snatch bed dead night bfg fear wor...
rating_5_42407,"[0.00014276373258326203, 0.002246614545583725,...",rating_5,center,babar establish celestevil beauti happi citi q...


С целью визуализации полученного пространства эмбеддингов с помощью алгоритма t-SNE воспользуемся методом `show_tsne()`:

In [ ]:
ues.show_tsne()

In [ ]:
ues.show_tsne(is_text_embeddings=True)

Также выведем облака слов книжных аннотаций для каждого вида взаимодействия, на которых размер слова пропорционален частоте его появления в каждой из категорий (чем больше размер слова, тем больше его частота):

In [ ]:
ues.show_wordclouds()